# Notebook de test des données lapi post-traitees

In [2]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin\git\Outils\Outils\Martin_Perso')
import donnees_posttraitees as t
import pandas as pd
import geopandas as gpd
import numpy as np
import altair as alt
import datetime as dt
pd.set_option('display.max_colwidth', 1000)
import os, math
from sklearn.cluster import DBSCAN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. import des données, traitement de mise en forme
## 1.1 Import
Les données sont importées sur la période du 24/01/19 au 23/02/19, sur laquelle il n'y a pas de manque de données à priori

In [ ]:
df_passages, df_plaques, df_immat=t.ouvrir_fichier_lapi_final('2019-01-24 00:00:00','2019-02-13 23:59:59')

## 1.2. Suppression des doublons

In [9]:
df_passages=t.supprimer_doublons(df_passages)

## 1.2 Ajout de l'attribut de différenciation VL / PL

In [14]:
df_passages2=t.affecter_type(df_passages,df_immat)

***ATTENTION : ENCORE 18764 IMMATRCULATION COURTE PRESENTES DANS TE_PASSAGES ET NON PRENSENTES DANS TE_PLAQUES_COURTE :***

In [ ]:
df_passages2.loc[df_passages2.immatriculation.isna()]

***ATTENTION : PLUS DE 1M D'IMMATRCULATION SANS VALEUR DE VL_TOT ou PL_TOT ou VUL_TOT :***

In [19]:
df_passages2.loc[df_passages2['type_veh']==-1]

In [ ]:
df_immat.loc[df_immat['immatriculation']=='000013c5f1fc019be0ac5a49a32f675be089542a']

1266400